In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc 

In [ ]:
# Use COVET feature maps to represent niches
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/covet_outputs/tonsil_covet.h5ad')
adata

AnnData object with n_obs × n_vars = 5778 × 3549
    obs: 'cell_type', 'author_cell_type', 'cell_type_int', 'leiden', 'leiden_R', 'cell_type_2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'CovGenes', 'author_cell_type_colors', 'cell_thresholds', 'cell_type_2_colors', 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_R', 'leiden_colors', 'ligand_receivers', 'neighbors', 'pca', 'received_ligands_tfl', 'umap'
    obsm: 'COVET', 'COVET_SQRT', 'DC_COVET', 'FDL_COVET', 'X_pca', 'X_umap', 'ora_estimate', 'ora_pvals', 'spatial', 'spatial_unscaled'
    varm: 'PCs'
    layers: 'imputed_count', 'log', 'normalized_count'
    obsp: 'connectivities', 'distances'

In [ ]:
import sys 
sys.path.append('../../src')

from spaceoracle.models.adapted_estimators import PrefeaturizedCellularProgramsEstimator
from spaceoracle.tools.network import RegulatoryFactory

In [ ]:
co_grn = RegulatoryFactory(
    colinks_path='/ix/djishnu/shared/djishnu_kor11/training_data_2025/snrna_human_tonsil_colinks.pkl',
    # colinks_path='/Users/koush/Desktop/training_data/snrna_human_tonsil_colinks.pkl',
    annot='cell_type_int'
)

In [ ]:
estimator = PrefeaturizedCellularProgramsEstimator(
    adata, target_gene='PAX5', cluster_annot='cell_type_int',
    radius=200, contact_distance=30, grn=co_grn, sp_maps_key='COVET_SQRT')

In [ ]:
estimator.fit(
    num_epochs=10, 
    learning_rate=5e-3, 
    score_threshold=0.2,
    estimator='lasso'
)

In [ ]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

spatial_estimator = SpatialCellularProgramsEstimator(
    adata, target_gene='PAX5', cluster_annot='cell_type_int',
    radius=200, contact_distance=30, grn=co_grn)

spatial_estimator.fit(
    num_epochs=10, 
    learning_rate=5e-3, 
    score_threshold=0.2,
    estimator='lasso'
)